Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.60301700 0.267258057 0.00422572091 0.0042825428 1000
 beta   0.90518034 0.043246744 0.00068379105 0.0007694696 1000
sigma   5.10826989 0.189826578 0.00300142173 0.0024240672 1000

Quantiles:
          2.5%       25.0%       50.0%        75.0%        97.5%   
alpha 154.0820000 154.4230000 154.6005000 154.79000000 155.13102500
 beta   0.8197474   0.8762585   0.9045965   0.93411475   0.99256603
sigma   4.7561248   4.9775050   5.1018500   5.23448750   5.49705125



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.725 0.865136 5.08905; 154.694 0.953491 5.05677; … ; 154.864 0.898148 4.91997; 154.503 0.885442 5.41598]

Union{Missing, Float64}[154.81 0.87047 5.29366; 154.408 0.875542 5.12124; … ; 154.094 0.911889 5.46741; 154.478 0.908087 5.39037]

Union{Missing, Float64}[154.807 0.866999 5.09479; 154.643 0.850079 5.19143; … ; 154.784 0.893493 4.99226; 154.604 0.891352 4.92604]

Union{Missing, Float64}[154.805 0.973108 5.1372; 154.775 0.915245 5.1418; … ; 154.348 0.83654 5.24703; 154.231 0.886438 5.30987]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
alpha 154.60301700 0.267258057 0.00422572091 0.0042825428 1000
 beta   0.90518034 0.043246744 0.00068379105 0.0007694696 1000
sigma   5.10826989 0.189826578 0.00300142173 0.0024240672 1000

Quantiles:
          2.5%       25.0%       50.0%        75.0%        97.5%   
alpha 154.0820000 154.4230000 154.6005000 154.79000000 155.13102500
 beta   0.8197474   0.8762585   0.9045965   0.93411475   0.99256603
sigma   4.7561248   4.9775050   5.1018500   5.23448750   5.49705125



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*